The code below defines the neural network class for a 3 layer neural network.

In [201]:
#Import statements

import numpy as np
import scipy.special
import inspect

In [202]:
# The definition of the neural network class.

class myNeuralNetwork:
    
    #Below is the code for the constructor which 
    #initialise the nerual network object.
    
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # Our constructor function takes as input the number of nodes in the input
        # layer, hidden layer, and the output layer. We also take as input the 
        # learning rate.
        
        # We assign them to the variables inputNodes, hiddenNodes, outputNodes, 
        # learningRate.
        
        self.input_nodes_field = input_nodes
        self.hidden_nodes_field = hidden_nodes
        self.output_nodes_field = output_nodes
        self.learning_rate_field = learning_rate
        
        
        
        # Now we have to create the network of links and weights.
        # We express the link-weights as a matrix. The matrix containining the input_layer - hidden_layer 
        # links is of size hidden_nodes by input_nodes. The other matrix is the hidden_layer - output_layer 
        #which is of size output_nodes by hidden_nodes weights inside the arrays are w_i_j, where link 
        #is from node i to node j in the next layer

        self.weights_input_hidden = np.random.normal(0.0, pow(self.input_nodes_field, -0.5), (self.hidden_nodes_field, self.input_nodes_field))
        self.weights_hidden_output = np.random.normal(0.0, pow(self.output_nodes_field, -0.5), (self.output_nodes_field, self.hidden_nodes_field))
        
        
        #Getting the activation function, which in our case will simply be the sigmoid function.
        
        self.activationFunction = lambda x: scipy.special.expit(x)
        
    
    #Below is the code for the function which trains the neural network.
    
    def train(self, input_list, target_list):
        inputs = np.array(input_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T
        
        #Inputs into hidden layer.
        inputs_hidden_layer = np.dot(self.weights_input_hidden, inputs)
        #outputs from hidden layer.
        outputs_hidden_layer = self.activationFunction(inputs_hidden_layer)
        
        
        #Inputs into output layer.
        inputs_output_layer = np.dot(self.weights_hidden_output, outputs_hidden_layer)
        #final output
        final_outputs = self.activationFunction(inputs_output_layer)
        
        #calculate the output error.
        output_layer_errors = targets - final_outputs
        #calculate the errors for the hidden layer.
        hidden_layer_errors = np.dot(self.weights_hidden_output.T, output_layer_errors)
        
        ##Updating the weights. The actual learning!
        
        #Updating the wights from the hidden layer to the output layer.
        
        self.weights_hidden_output += self.learning_rate_field*np.dot(
            ((output_layer_errors)*final_outputs*(1 - final_outputs)), np.transpose(outputs_hidden_layer)
        )
        self.weights_input_hidden += self.learning_rate_field*np.dot(
            ((hidden_layer_errors)*outputs_hidden_layer*(1 - outputs_hidden_layer)), np.transpose(inputs)
        )
        pass
    
    # Below is the code for the function which helps us give input into the 
    # neural network and get outputs
    
    def predict(self, input_list):
        
        #Conversion into 2-d array for multiplication using numpy.
        inputs = np.array(input_list, ndmin=2).T
        
        #Inputs into hidden layer.
        inputs_hidden_layer = np.dot(self.weights_input_hidden, inputs)
        #outputs from hidden layer.
        outputs_hidden_layer = self.activationFunction(inputs_hidden_layer)
        
        
        #Inputs into output layer.
        inputs_output_layer = np.dot(self.weights_hidden_output, outputs_hidden_layer)
        #final output
        return_value = self.activationFunction(inputs_output_layer)
        
        
        return return_value
    
    

In [203]:
#loading the training data.

data_handler = open("mnist_train.csv", 'r')
data_list = data_handler.readlines()
data_handler.close()

In [204]:
#Creating the neural network object.

input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.2

n = myNeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [205]:
#Training the neural network object using the data set that was loaded.

#Setting value for epochs
epochs = 2
for i in range(epochs):
    for record in data_list:

        #Splits the string into a list of string elements using the comma.
        values = record.split(',')

        #Scaling and shifting the inputs so that they could be fed into our activation 
        #function which happens to be a sigmoid function.

        scaled_shifted_inputs = (np.asfarray(values[1:])/255.0 * 0.99) + 0.01

        #Creating the target_values array for each node in the output layer.
        target_value_array = np.zeros(output_nodes) + 0.01
        target_value_array[int(values[0])] = 0.99

        #Updating the weights for that particular label.
        #print(target_value_array)
        #print(len(target_value_array))

        n.train(scaled_shifted_inputs, target_value_array)
        pass

In [208]:
#Testing the neural network
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

score_card = []

for record in test_data_list:
    values= record.split(',')
    actual_label = int(values[0])
    scaled_inputs = (np.asfarray(values[1:])/255.0 * 0.99)
    output_layer_array = n.predict(scaled_inputs)
    predicted_label = np.argmax(output_layer_array)
    
    if (actual_label == predicted_label):
        score_card.append(1)
    else:
        score_card.append(0)
        
#Calculating the performace of the neural_network on the training data.
score_card_array = np.asarray(score_card)
sum = score_card_array.sum()
performance = sum/len(score_card)
print(performance)



0.9682
